In [149]:
# --- initial set up ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
import time
from tqdm import tqdm


In [150]:
# --- Redefining Inputs ---


def generate_synthetic_prices(S0 = [50, 25, 1, 200, 13500, 200],
commodities = ['crude', 'natural', 'copper', 'corn', 'gold', 'silver'], 
n_steps = 1000, 
seed = 42, 
mu = [0.07, 0.05, 0.06, 0.04, 0.03, 0.025], 
sigma = [0.25, 0.40, 0.30, 0.20, 0.15, 0.18], 
rho_ij = 0.30):

    for i in tqdm(range(n_steps), desc = "Generating Commodity Prices"):

        np.random.seed(seed)
        n_assets = len(commodities)
        S = np.zeros((n_steps, n_assets))
        rho = np.full((n_assets, n_assets), rho_ij)
        np.fill_diagonal(rho, 1.0)

        # --- Performing Cholesky Decomposition ---

        L = np.linalg.cholesky(rho)

        daily_mu = np.array(mu) / 252
        daily_vol = np.array(sigma) / np.sqrt(252)
        Z = np.random.standard_normal((n_steps, n_assets))
        rho_shocks = Z @ L.T
        returns = daily_mu + daily_vol * rho_shocks

        # --- Generate Price Process ---

        S[0] = S0
        for t in range(1, n_steps):
            S[t] = S[t-1] * np.exp(returns[t])

        # --- Commodity Specific Adjustments ---

        S[:, 3] *= (1 + 0.05 * np.sin(2 * np.pi * t / 250))
        S[:, 4] += np.random.normal(0, 5, n_steps)
        S[:, 5] += np.random.normal(0, 0.5, n_steps)

        dates = pd.bdate_range("2020-01-01", periods = n_steps)
        S_df = pd.DataFrame(S, index = dates, columns = commodities)
        S_df
        S_df.to_csv("/Users/deborahakintoye/GitHub/quant-strat-lab/projects/25_cross_sectional_commodity_momentum_analysis/commodity_prices.csv")
    print(f"Synthetic Price Generation Complete.")

    

In [151]:
generate_synthetic_prices();

Generating Commodity Prices: 100%|██████████| 1000/1000 [00:40<00:00, 24.88it/s]

Synthetic Price Generation Complete.


In [ ]:
# --- Factor Construction Tools ---

def compute_carry(window = 90):
    window = window
    carry = [f"{c}_carry" for c in commodities]
    S_df[carry] = ((S_df[commodities] - S_df[commodities].shift(window)) / S_df[commodities].shift(window)).fillna(0.0)
    return S_df[carry]

def compute_momentum(window = 250):
    window = window
    momentum = [f"{c}_momentum" for c in commodities]
    S_df[momentum] = ((S_df[commodities] / S_df[commodities].shift(window)) - 1).fillna(0.0)
    return S_df[momentum]

def compute_mean_reversion(window = 50):
    window = window
    mr = [f"{c}_mr" for c in commodities]
    ma = S_df[commodities].rolling(window = window).mean()
    S_df[mr] = - zscore((S_df[commodities] - ma).fillna(0.0))
    return S_df[mr]


In [153]:

compute_carry()
compute_momentum()
compute_mean_reversion()
S_df

,crude,natural,copper,corn,gold,silver,crude_carry,natural_carry,copper_carry,corn_carry,...,copper_mr,corn_mr,gold_mr,silver_mr,crude_z,natural_z,copper_z,corn_z,gold_z,silver_z
2020-01-01,50.000000,25.000000,1.000000,199.748699,13498.878795,199.928288,0.000000,0.000000,0.000000,0.000000,...,0.346030,-0.155517,0.093191,0.355893,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-02,50.094077,25.941679,1.001561,202.940726,13525.442475,201.833753,0.000000,0.000000,0.000000,0.000000,...,0.346030,-0.155517,0.093191,0.355893,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-03,50.018459,25.889782,0.994318,205.842027,13465.563898,200.377574,0.000000,0.000000,0.000000,0.000000,...,0.346030,-0.155517,0.093191,0.355893,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-06,51.731785,26.939255,0.969277,205.670164,13298.740985,199.220276,0.000000,0.000000,0.000000,0.000000,...,0.346030,-0.155517,0.093191,0.355893,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-07,49.250682,26.226364,0.932989,202.831239,12982.948231,195.512508,0.000000,0.000000,0.000000,0.000000,...,0.346030,-0.155517,0.093191,0.355893,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,42.278064,29.300443,1.940187,156.091781,13859.210939,355.878890,-0.047308,0.019714,0.322221,-0.030500,...,-2.689440,-0.347441,-1.275976,-0.079257,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-26,42.880299,29.739099,1.942294,157.090593,14069.498901,356.150471,0.009865,0.036090,0.318230,-0.013400,...,-2.597509,-0.479084,-1.717522,-0.078740,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-27,43.165067,29.785187,1.961260,156.291883,14096.117016,359.858934,0.015290,0.028571,0.256219,-0.041470,...,-2.737646,-0.408747,-1.719945,-0.407949,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-30,42.544596,28.650520,1.897105,152.684944,14029.531860,351.700996,-0.001841,0.007621,0.200524,-0.065334,...,-1.765049,-0.015854,-1.511293,0.377971,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
# --- Portfolio Construction Tools ---

def z_score_cross_section():
    z_scores = [f"{c}_z" for c in commodities]
    S_df[z_scores] = S_df.groupby(dates)[commodities].transform(zscore).fillna(0.0)
    return S_df[z_scores]


In [155]:
z_score_cross_section()

,crude_z,natural_z,copper_z,corn_z,gold_z,silver_z
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-06,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-07,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2023-10-25,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-26,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-27,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-30,0.0,0.0,0.0,0.0,0.0,0.0


NameError: name 'momentum' is not defined